In [1]:
require("/home/amit/Downloads/SemVII/QuBase.jl/src/QuBase.jl")
using QuBase
require("/home/amit/Downloads/SemVII/QuDynamics.jl/src/QuDynamics.jl")
using QuDynamics
using BenchmarkLite

In [2]:
type SampleProc{Alg} <: Proc end

Jaynes-Cummings type which takes the configuration parameters.
These parameters are used for benchmarking, creating various
JaynesCummings types with various parameters and using
the solvers to get the timing results.

In [3]:
type JaynesCummings
    N::Int
    wc::Float64
    wa::Float64
    g::Float64
    kappa::Float64
    gamma::Float64
    use_rwa::Bool
    tlist
end

Name of the solver, used in the creating the table of timing analysis

In [4]:
Base.string{Alg}(::SampleProc{Alg}) = lowercase(string(Alg))

string (generic function with 9 methods)

Constructing a valid configuration, this is set to `true` as we have
set the type of the required fields in the construct `JaynesCummings` itself.

In [5]:
Base.isvalid{Alg}(p::SampleProc{Alg}, cfg::JaynesCummings) = true

isvalid (generic function with 23 methods)

This returns the `hamitlonian`, `collapse operators`, `initial density matrix`,
`observable whose expectation value is to be calculated` taking in the `cfg`
configuration of the system.

In [6]:
function operator_set(p::SampleProc, cfg::JaynesCummings)
    idc = QuArray(eye(cfg.N))
    ida = QuArray(eye(2))
    a  = tensor(lowerop(cfg.N), ida)
    sm = tensor(idc, lowerop(2))
    if cfg.use_rwa
        H = cfg.wc * a' * a + cfg.wa * sm' * sm + cfg.g * (a' * sm + a * sm')
    else
        H = cfg.wc * a' * a + cfg.wa * sm' * sm + cfg.g * (a' + a) * (sm + sm')
    end
    c_op_list = Array(QuBase.AbstractQuMatrix, 0)
    n_th_a = 0.0
    rate_1 = cfg.kappa * (1 + n_th_a)
    if rate_1 > 0.0
        push!(c_op_list, full(sqrt(rate_1) * a))
    end
    rate = cfg.kappa * n_th_a
    if rate > 0.0
        push!(c_op_list, full(sqrt(rate) * a'))
    end
    rate = cfg.gamma
    if rate > 0.0
        push!(c_op_list, full(sqrt(rate) * sm))
    end
    psi = complex(tensor(statevec(1, FiniteBasis(cfg.N)), statevec(2, FiniteBasis(2))))
    return full(H), c_op_list, psi*psi', a*a'
end

operator_set (generic function with 1 method)

Start states are pre-allocated, that is they are constructed before the evaluation
here we return the `hamiltonian`, `collapse operator list`, `initial density matrix`,
`observable whose expectation is to be calculated` using the above function
i.e., operator_set. The first three remain fixed, the fourth can be varied as required.

In [7]:
Base.start(p::SampleProc, cfg::JaynesCummings) = (operator_set(p, cfg))

start (generic function with 37 methods)

In [8]:
Base.length(p::SampleProc, cfg::JaynesCummings) = cfg.N

length (generic function with 50 methods)

This function gets the timing done for calculation of trace, using the pre-allocated
constructions from the start method i.e., the hamiltonain, collapse operator list,
initial density matrix and observable whose expectation is to be calculated.

In [9]:
function Base.run{Alg}(p::SampleProc{Alg}, cfg::JaynesCummings, s::(QuBase.QuArray, Array, QuBase.QuArray, QuBase.QuArray))
    qprop = QuPropagator(s[1], s[2], s[3], cfg.tlist, Alg())
    for (t, psi) in qprop
        trace(coeffs(psi*(s[4])))
    end
end

run (generic function with 4 methods)

In [10]:
Base.done(p::SampleProc, cfg, s) = nothing

done (generic function with 40 methods)

Creating the array of methods on which benchmarks have to be performed

In [11]:
procs = Proc[ SampleProc{QuExpmV}(),
              SampleProc{QuExpokit}(),
              SampleProc{QuODE45}(),
              SampleProc{QuODE78}(),
              SampleProc{QuODE23s}()]

5-element Array{Proc,1}:
 SampleProc{QuExpmV}()  
 SampleProc{QuExpokit}()
 SampleProc{QuODE45}()  
 SampleProc{QuODE78}()  
 SampleProc{QuODE23s}() 

In [12]:
# Setting up various configurations.
cfgs = JaynesCummings[JaynesCummings(2, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 0.:0.1:2*pi),
                      JaynesCummings(3, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 0.:0.1:2*pi),
                      JaynesCummings(6, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 0.:0.1:2*pi),
                      JaynesCummings(10, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 0.:0.1:2*pi),
                      JaynesCummings(2, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, false, 0.:0.1:2*pi),
                      JaynesCummings(3, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, false, 0.:0.1:2*pi),
                      JaynesCummings(6, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, false, 0.:0.1:2*pi),
                      JaynesCummings(2, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 0.:0.1:2*pi),
                      JaynesCummings(3, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 0.0:0.20134228187919462:30.0),
                      JaynesCummings(6, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 0.0:0.20134228187919462:30.0),
                      JaynesCummings(10, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, true, 0.0:0.20134228187919462:30.0),
                      JaynesCummings(2, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, false, 0.0:0.20134228187919462:30.0),
                      JaynesCummings(3, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, false, 0.0:0.20134228187919462:30.0),
                      JaynesCummings(6, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, false, 0.0:0.20134228187919462:30.0),
                      JaynesCummings(10, 1.0 * 2 * pi, 1.0 * 2 * pi, 0.05 * 2 * pi, 0.05, 0.15, false, 0.0:0.20134228187919462:30.0)]

15-element Array{JaynesCummings,1}:
 JaynesCummings(2,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                   
 JaynesCummings(3,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                   
 JaynesCummings(6,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                   
 JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                  
 JaynesCummings(2,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,false,0.0:0.1:6.2)                  
 JaynesCummings(3,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,false,0.0:0.1:6.2)                  
 JaynesCummings(6,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,false,0.0:0.1:6.2)                  
 JaynesCummings(2,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                   
 Jay

Generating the timing analysis.

In [13]:
rtable = run(procs, cfgs)

Benchmarking quexpmv ...
  quexpmv with cfg = JaynesCummings(2,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2): nruns = 38, elapsed = 0.350288969 secs
  quexpmv with cfg = JaynesCummings(3,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2): nruns = 76, elapsed = 1.220361545 secs
  quexpmv with cfg = JaynesCummings(6,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2): nruns = 8, elapsed = 1.040863584 secs
  quexpmv with cfg = JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2): nruns = 2, elapsed = 1.838067047 secs
  quexpmv with cfg = JaynesCummings(2,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,false,0.0:0.1:6.2): nruns = 36, elapsed = 0.362003684 secs
  quexpmv with cfg = JaynesCummings(3,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,false,0.0:0.1:6.2): nruns = 28, elapsed = 0.499074135 secs
  quexpm

BenchmarkTable [unit = sec]
config                                                                                                                 |  quexpmv  quexpokit  quode45  quode78  quode23s  
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
JaynesCummings(2,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                    |   0.0092     0.0087   0.0068   0.0056    0.0407  
JaynesCummings(3,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                    |   0.0161     0.0204   0.0173   0.0148    0.1491  
JaynesCummings(6,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                    |   0.1301     0.1984   0.1530   0.1401    1.8680  
JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                 

Table display from the above method.

In [14]:
show(rtable)

BenchmarkTable [unit = sec]
config                                                                                                                 |  quexpmv  quexpokit  quode45  quode78  quode23s  
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------
JaynesCummings(2,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                    |   0.0092     0.0087   0.0068   0.0056    0.0407  
JaynesCummings(3,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                    |   0.0161     0.0204   0.0173   0.0148    0.1491  
JaynesCummings(6,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                    |   0.1301     0.1984   0.1530   0.1401    1.8680  
JaynesCummings(10,6.283185307179586,6.283185307179586,0.3141592653589793,0.05,0.15,true,0.0:0.1:6.2)                 